In [1]:
import pandas as pd
import calendar # to turn month abbrev into integer
from functools import reduce # to join multiple tables 
from fastai.imports import *

C:\Users\S\Anaconda3\lib\importlib\_bootstrap.py:219: RuntimeWarning: cymem.cymem.Pool size changed, may indicate binary incompatibility. Expected 48 from C header, got 64 from PyObject
  return f(*args, **kwds)
C:\Users\S\Anaconda3\lib\importlib\_bootstrap.py:219: RuntimeWarning: cymem.cymem.Address size changed, may indicate binary incompatibility. Expected 24 from C header, got 40 from PyObject
  return f(*args, **kwds)


In [11]:
def add_datepart(df, fldname, drop=True, time=False):
    """add_datepart converts a column of df from a datetime64 to many columns containing
    the information from the date. This applies changes inplace.
    Parameters:
    -----------
    df: A pandas data frame. df gain several new columns.
    fldname: A string that is the name of the date column you wish to expand.
        If it is not a datetime64 series, it will be converted to one with pd.to_datetime.
    drop: If true then the original date column will be removed.
    time: If true time features: Hour, Minute, Second will be added.
    Examples:
    ---------
    >>> df = pd.DataFrame({ 'A' : pd.to_datetime(['3/11/2000', '3/12/2000', '3/13/2000'], infer_datetime_format=False) })
    >>> df
        A
    0   2000-03-11
    1   2000-03-12
    2   2000-03-13
    >>> add_datepart(df, 'A')
    >>> df
        AYear AMonth AWeek ADay ADayofweek ADayofyear AIs_month_end AIs_month_start AIs_quarter_end AIs_quarter_start AIs_year_end AIs_year_start AElapsed
    0   2000  3      10    11   5          71         False         False           False           False             False        False          952732800
    1   2000  3      10    12   6          72         False         False           False           False             False        False          952819200
    2   2000  3      11    13   0          73         False         False           False           False             False        False          952905600
    """
    fld = df[fldname]
    fld_dtype = fld.dtype
    if isinstance(fld_dtype, pd.core.dtypes.dtypes.DatetimeTZDtype):
        fld_dtype = np.datetime64

    if not np.issubdtype(fld_dtype, np.datetime64):
        df[fldname] = fld = pd.to_datetime(fld, infer_datetime_format=True)
    targ_pre = re.sub('[Dd]ate$', '', fldname)
    attr = ['Year', 'Month', 'Week', 'Day', 'Dayofweek', 'Dayofyear',
            'Is_month_end', 'Is_month_start', 'Is_quarter_end', 'Is_quarter_start', 'Is_year_end', 'Is_year_start']
    if time: attr = attr + ['Hour', 'Minute', 'Second']
    for n in attr: df[targ_pre + n] = getattr(fld.dt, n.lower())
    df[targ_pre + 'Elapsed'] = fld.astype(np.int64) // 10 ** 9
    if drop: df.drop(fldname, axis=1, inplace=True)

In [19]:
data_dir = "dataset/"
group_table = pd.read_csv(data_dir + "group.csv")
airline_table = pd.read_csv(data_dir + "airline.csv")
order_table = pd.read_csv(data_dir + "order.csv")
dayschedule_table = pd.read_csv(data_dir + "day_schedule.csv")
train_set = pd.read_csv("training-set.csv")
test_set = pd.read_csv("testing-set.csv")

In [40]:
# first, check if any groups contain null value
whole_table = [group_table, airline_table, order_table, dayschedule_table]
null_group_id = {
    "group_table": [],
    "airline_table": [],
    "order_table": [],
    "dayschedule_table": []
}

for table_name, table in zip(null_group_id.keys(), whole_table):
    cur_null_col = table.columns[table.isnull().any()] # check if any columns contain null
    for col in cur_null_col:
        print(col + " contains null")
        null_group_id[table_name].extend(table[table[col].isnull() == True]["group_id"])
        
group_contain_null = []
for table_name in null_group_id.keys():
    group_contain_null.extend(null_group_id[table_name])
group_contain_null = set(group_contain_null) # deduplicate

""" although [product_name, promotion_prog] in projectgroup.csv contain null value
 but I will not use these two feature to build model in first stage
 so I can process it later."""

# these are the group_id all table has.
group_id_set = reduce(lambda x, y: x.intersection(set(y["group_id"])), whole_table[1:], set(whole_table[0]["group_id"]))
for table in whole_table:
    diff_set = set(table["group_id"]).difference(group_id_set)
    if not diff_set:
        continue
    drop_index = list(map(lambda x: table.index[table["group_id"] == x].tolist(), diff_set))
    for ind in drop_index[1:]:
        drop_index[0].extend(ind)
    drop_index = drop_index[0]
    table.drop(index=drop_index, inplace=True)
group_check = reduce(lambda x, y: x.difference(set(y["group_id"])), whole_table[1:], set(whole_table[0]["group_id"]))

product_name contains null
promotion_prog contains null


In [13]:
# convert them to standard datetime type so that I can use the tool fastai.add_datepart()
group_table.begin_date = pd.to_datetime(group_table.begin_date)
add_datepart(group_table, 'begin_date')

# convert them to integer code
for col in ["sub_line", "area"]:
    group_table[col] = list(map(lambda x: int(x[-1]), group_table[col]))

#show examples
group_table.head(50)

,group_id,sub_line,area,days,price,product_name,promotion_prog,begin_Year,begin_Month,begin_Week,begin_Day,begin_Dayofweek,begin_Dayofyear,begin_Is_month_end,begin_Is_month_start,begin_Is_quarter_end,begin_Is_quarter_start,begin_Is_year_end,begin_Is_year_start,begin_Elapsed
0,63695,1,1,12,106900,最高省8000》大美西～夏威夷、優勝美地、西峽天空步道、聖地牙哥、環球影城(哈利波特)12日,&lt;B&gt;[GG]《行銷活動》[//]&lt;/B&gt;&lt;font face...,2017,3,12,22,2,81,False,False,False,False,False,False,1490140800
1,53481,1,1,12,112900,《玩樂369》旗艦大美西12日～升等五星、優勝美地、西峽天空步道、卡利哥鬼鎮、環球影城大美西12日,&lt;B&gt;[GG]《行銷活動》[//]&lt;/B&gt;&lt;font face...,2017,5,20,17,2,137,False,False,False,False,False,False,1494979200
2,54305,1,1,12,115900,《玩樂369》旗艦大美西12日～升等五星、優勝美地、西峽天空步道、卡利哥鬼鎮、環球影城大美西12日,&lt;B&gt;[GG]《行銷活動》[//]&lt;/B&gt;&lt;font face...,2017,6,25,21,2,172,False,False,False,False,False,False,1498003200
3,41766,1,1,12,126900,《玩樂369》旗艦大美西12日～升等五星、優勝美地、西峽天空步道、卡利哥鬼鎮、環球影城大美西12日,&lt;B&gt;[GG]《行銷活動》[//]&lt;/B&gt;&lt;font face...,2017,7,28,12,2,193,False,False,False,False,False,False,1499817600
4,32196,1,1,12,126900,《玩樂369》旗艦大美西12日～升等五星、優勝美地、西峽天空步道、卡利哥鬼鎮、環球影城大美西12日,&lt;B&gt;[GG]《行銷活動》[//]&lt;/B&gt;&lt;font face...,2017,8,32,9,2,221,False,False,False,False,False,False,1502236800
5,25423,1,1,12,116900,《玩樂369》旗艦大美西12日～升等五星、優勝美地、西峽天空步道、卡利哥鬼鎮、環球影城大美西12日,&lt;B&gt;[GG]《行銷活動》[//]&lt;/B&gt;&lt;font face...,2017,9,39,27,2,270,False,False,False,False,False,False,1506470400
6,25705,1,1,12,116900,《玩樂369》旗艦大美西12日～升等五星、優勝美地、西峽天空步道、卡利哥鬼鎮、環球影城大美西12日,&lt;B&gt;[GG]《行銷活動》[//]&lt;/B&gt;&lt;font face...,2017,12,50,13,2,347,False,False,False,False,False,False,1513123200
7,44047,1,1,12,116900,《玩樂369》旗艦大美西12日～升等五星、優勝美地、西峽天空步道、卡利哥鬼鎮、環球影城大美西12日,&lt;B&gt;[GG]《行銷活動》[//]&lt;/B&gt;&lt;font face...,2017,11,47,22,2,326,False,False,False,False,False,False,1511308800
8,21406,1,1,12,116900,《玩樂369》旗艦大美西12日～升等五星、優勝美地、西峽天空步道、卡利哥鬼鎮、環球影城大美西12日,&lt;B&gt;[GG]《行銷活動》[//]&lt;/B&gt;&lt;font face...,2017,10,41,11,2,284,False,False,False,False,False,False,1507680000
9,63785,1,2,10,70900,《加999升等豪經艙》驚奇美西～大峽谷天空步道、賭城摩天輪、美食饗宴、環球影城、迪士尼10日,"&lt;font face=""微軟正黑體"" size=""2""&gt;[GG]&lt;b&gt...",2017,1,1,7,5,7,False,False,False,False,False,False,1483747200


In [ ]:
"""
# split the "date" feature into day and month
month_int = {}
for i, month in enumerate(calendar.month_abbr): # use this dictionary to map month abbrev to integer
    month_int[month] = i

for col, table in zip(["begin_date", "order_date"], [group_table, order_table]):
    col_head = col.split("_")[0]
    new_bg_date = table[col].str.split("-", expand=True) # [day, month, year]
    new_bg_date[1] = list(map(lambda x: month_int[x], new_bg_date[1]))
    table[col_head + "_" + "day"] = pd.to_numeric(new_bg_date[0])
    table[col_head + "_" + "month"] = new_bg_date[1]
    table.drop(columns=[col], inplace=True)
"""

In [14]:
# convert them to integer code
for col in ["source_1", "source_2", "unit"]:
    order_table[col] = list(map(lambda x: int(x[-1]), order_table[col]))

# process the datetime feature
order_table.order_date = pd.to_datetime(order_table.order_date)
add_datepart(order_table, "order_date")

# show examples
order_table.head()

,order_id,group_id,source_1,source_2,unit,people_amount,order_Year,order_Month,order_Week,order_Day,order_Dayofweek,order_Dayofyear,order_Is_month_end,order_Is_month_start,order_Is_quarter_end,order_Is_quarter_start,order_Is_year_end,order_Is_year_start,order_Elapsed
0,136100,63695,1,1,1,2,2017,1,1,5,3,5,False,False,False,False,False,False,1483574400
1,140370,63695,1,1,2,1,2016,11,45,9,2,314,False,False,False,False,False,False,1478649600
2,211009,63695,1,1,3,2,2017,1,3,20,4,20,False,False,False,False,False,False,1484870400
3,41571,53481,1,1,4,4,2017,3,11,14,1,73,False,False,False,False,False,False,1489449600
4,64125,53481,2,1,5,1,2017,3,13,27,0,86,False,False,False,False,False,False,1490572800


In [15]:
# split the [date time] into date and time
# then split the time into hour and minute
for col in airline_table.columns[1:]:
    col_head = col.split("_")
    col_head = col_head[0] + "_" + col_head[1]
    new_col = airline_table[col].str.split(" ", expand=True)
    airline_table[col_head + "_" + "date"] = new_col[0]
    hour_minute = new_col[1].str.split(":", expand=True)
    airline_table[col_head + "_" + "hour"] = pd.to_numeric(hour_minute[0])
    airline_table[col_head + "_" + "minute"] = pd.to_numeric(hour_minute[1])
    airline_table.drop(columns=[col], inplace=True)

KeyError: 1

In [44]:
# convert them to standard datetime type so that I can use the tool fastai.add_datepart()
for date_col in airline_table.columns[1:]:
    if "date" in date_col:
        airline_table[date_col] = pd.to_datetime(airline_table[date_col])
        add_datepart(airline_table, date_col)

# show examples
airline_table.head()

,group_id,abroad_fly_hour,abroad_fly_minute,abroad_arrive_hour,abroad_arrive_minute,home_fly_hour,home_fly_minute,home_arrive_hour,home_arrive_minute,abroad_fly_Year,...,home_arrive_Day,home_arrive_Dayofweek,home_arrive_Dayofyear,home_arrive_Is_month_end,home_arrive_Is_month_start,home_arrive_Is_quarter_end,home_arrive_Is_quarter_start,home_arrive_Is_year_end,home_arrive_Is_year_start,home_arrive_Elapsed
0,2,17,0,20,25,18,25,20,45,2017,...,27,0,331,False,False,False,False,False,False,1511740800
1,3,17,50,20,30,11,55,13,55,2017,...,22,6,22,False,False,False,False,False,False,1485043200
2,4,14,40,18,30,19,30,21,50,2017,...,1,1,213,False,True,False,False,False,False,1501545600
3,7,10,55,14,40,15,50,19,20,2017,...,3,5,154,False,False,False,False,False,False,1496448000
4,9,18,10,21,35,21,5,22,55,2017,...,16,0,16,False,False,False,False,False,False,1484524800


In [18]:
# convert airline_table to a table contains "abroad_fly_time", "abroad_arrive_time", "home_fly_time", "home_arrive_time"
group_id_set = set(airline_table.group_id)
new_col_name = ["group_id", "abroad_fly_time", "abroad_arrive_time","home_fly_time", "home_arrive_time"]
airline_table.drop(columns=["go_back", "src_airport", "dst_airport"], inplace=True)
tmp_data = list(map(lambda x: airline_table[airline_table.group_id == x].iloc[0], group_id_set)) # go abroad
new_airline_table = reduce(lambda x, y: pd.concat([x, y.to_frame().T]), tmp_data[1:], tmp_data[0].to_frame().T)
tmp_data = list(map(lambda x: airline_table[airline_table.group_id == x].iloc[-1], group_id_set)) # go home
tmp_table = reduce(lambda x, y: pd.concat([x, y.to_frame().T]), tmp_data[1:], tmp_data[0].to_frame().T)
new_airline_table = pd.DataFrame.merge(new_airline_table, tmp_table, on="group_id")
new_airline_table.columns = new_col_name
new_airline_table.to_csv("dataset/new_airline.csv", encoding="utf-8", index=False)

# split the "date" feature into day and month
# and split the "time" feature into hour and minute
"""
for new_col, col in zip(["fly", "arrive"], ["fly_time", "arrive_time"]):
    new_time = airline_table[col].str.split(" ", expand=True)

    date = new_time[0]
    date = date.str.split("/", expand=True) # [year, month, day]
    airline_table[new_col + "_" + "month"] = pd.to_numeric(date[1])
    airline_table[new_col + "_" + "day"] = pd.to_numeric(date[2])

    time = new_time[1]
    time = time.str.split(":", expand=True) # [hour, minute]
    airline_table[new_col + "_" + "hour"] = pd.to_numeric(time[0])
    airline_table[new_col + "_" + "minute"] = pd.to_numeric(time[1])

airline_table.drop(columns=["fly_time", "arrive_time"], inplace=True)
"""

# map go_back to binary value, 去程=0 回程=1
# airline_table["go_back"] = list(map(lambda x: {"去程": 0, "回程":1}[x], airline_table["go_back"]))

# convert the airport name to integer code
"""
all_airport = []
for col in ["src_airport", "dst_airport"]:
    all_airport.extend(airline_table[col].tolist())
all_airport = list(set(all_airport))
airport_code = {}
for i, airport in enumerate(all_airport):
    airport_code[airport] = i
for col in ["src_airport", "dst_airport"]:
    airline_table[col] = list(map(lambda x: str(airport_code[x]), airline_table[col]))
    airline_table[col] = pd.to_numeric(airline_table[col])
"""

ValueError: labels ['go_back' 'src_airport' 'dst_airport'] not contained in axis

In [13]:
dayschedule_table.head(10)

,group_id,day,title
0,63695,1.0,台北 [AIR]歐胡島市區觀光【州政府．皇宮．國王銅像．中國城】
1,63695,2.0,夏威夷小環島【鑽石頭山．恐龍灣．噴泉口】－珍珠港[BOATS]豪華愛之船Star of Ho...
2,63695,3.0,夏威夷 [AIR] 舊金山
3,63695,4.0,舊金山市區觀光【藝術宮．金門大橋．金門公園．漁人碼頭．金門灣遊船．市政廳．雙子峰．九曲花街】
4,63695,5.0,舊金山－300KM－優勝美地國家公園－100KM－佛雷斯諾
5,63695,6.0,佛雷斯諾－380KM－巴斯托《名牌大賣場》－260KM－拉斯維加斯
6,63695,7.0,拉斯維加斯－200KM－大峽谷【西峽「天空步道」一日遊】－200KM－拉斯維加斯
7,63695,8.0,拉斯維加斯─450KM─洛杉磯市區觀光【中國戲院．星光大道．好萊塢杜比戲院 Dolby Th...
8,63695,9.0,洛杉磯190KM－聖地牙哥【海景火車－[BOATS]聖地牙哥港灣遊船－舊城區】
9,63695,10.0,洛杉磯【環球影城製片廠(哈利波特主題樂園)】


In [45]:
# check if any columns need to be convert to integer
for table_name, table in zip(null_group_id.keys(), whole_table):
    for col in table.columns:
        if type(table[col][0]) == str:
            print(table_name + ": " + col + " is string.")

group_table: product_name is string.
group_table: promotion_prog is string.
dayschedule_table: title is string.


In [46]:
processed_datadir = data_dir + "processed_data" + "/"
for table_name, table in zip(["group.csv", "order.csv", "airline.csv"], [group_table, order_table, airline_table]):
    table.to_csv(processed_datadir + "processed_" + table_name, encoding="utf-8", index=False)

In [48]:
main_table = pd.merge(train_set, order_table, on="order_id")
dfs = [main_table, group_table, airline_table]
main_table = reduce(lambda left, right: pd.merge(left, right, on="group_id"), dfs)
main_table.drop(columns=["product_name", "promotion_prog"], inplace=True)
main_table.head()

,order_id,deal_or_not,group_id,source_1,source_2,unit,people_amount,order_Year,order_Month,order_Week,...,home_arrive_Day,home_arrive_Dayofweek,home_arrive_Dayofyear,home_arrive_Is_month_end,home_arrive_Is_month_start,home_arrive_Is_quarter_end,home_arrive_Is_quarter_start,home_arrive_Is_year_end,home_arrive_Is_year_start,home_arrive_Elapsed
0,41571,1,53481,1,1,4,4,2017,3,11,...,28,6,148,False,False,False,False,False,False,1495929600
1,64125,1,53481,2,1,5,1,2017,3,13,...,28,6,148,False,False,False,False,False,False,1495929600
2,76131,0,53481,2,2,6,2,2017,4,16,...,28,6,148,False,False,False,False,False,False,1495929600
3,238671,0,53481,2,2,6,2,2017,4,16,...,28,6,148,False,False,False,False,False,False,1495929600
4,93305,1,41766,2,1,5,1,2017,5,18,...,23,6,204,False,False,False,False,False,False,1500768000


In [51]:
main_table.to_csv(processed_datadir + "main_table.csv", encoding="utf-8", index=False)

In [22]:
processed_datadir = data_dir + "processed_data" + "/"
order_table = pd.read_csv(processed_datadir + "processed_order.csv")
group_table = pd.read_csv(processed_datadir + "processed_group.csv")
airline_table = pd.read_csv(processed_datadir + "processed_airline.csv")

In [23]:
test_table = pd.merge(test_set, order_table, on="order_id")
dfs = [test_table, group_table, airline_table]
test_table = reduce(lambda left, right: pd.merge(left, right, on="group_id"), dfs)
test_table.drop(columns=["product_name", "promotion_prog"], inplace=True)
test_table.head()

,order_id,deal_or_not,group_id,source_1,source_2,unit,people_amount,order_Year,order_Month,order_Week,...,home_arrive_Day,home_arrive_Dayofweek,home_arrive_Dayofyear,home_arrive_Is_month_end,home_arrive_Is_month_start,home_arrive_Is_quarter_end,home_arrive_Is_quarter_start,home_arrive_Is_year_end,home_arrive_Is_year_start,home_arrive_Elapsed
0,136100,NaN,63695,1,1,1,2,2017,1,1,...,2,6,92,False,False,False,False,False,False,1491091200
1,140370,NaN,63695,1,1,2,1,2016,11,45,...,2,6,92,False,False,False,False,False,False,1491091200
2,211009,NaN,63695,1,1,3,2,2017,1,3,...,2,6,92,False,False,False,False,False,False,1491091200
3,5045,NaN,54305,1,1,7,6,2017,1,3,...,2,6,183,False,False,False,False,False,False,1498953600
4,37230,NaN,54305,2,3,6,2,2017,5,19,...,2,6,183,False,False,False,False,False,False,1498953600


In [20]:
test_table.to_csv(processed_datadir + "test_table.csv", encoding="utf-8", index=True)